In [ ]:
# importing the useful libraries
import os
import pandas as pd
import re
import nltk
import string
from nltk import pos_tag
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import ward, dendrogram
from gensim import corpora, models, similarities 
from nltk.stem.snowball import SnowballStemmer
import numpy as np
from itertools import chain
from nltk.corpus import stopwords
from sklearn import feature_extraction
from sklearn.cluster import KMeans
import pprint
import sys
import tempfile
from jinja2 import Template
from flask import Flask, render_template, flash, request, send_file
from wtforms import Form, TextField, TextAreaField, validators, StringField, SubmitField
from jinja2 import Environment
from jinja2.loaders import FileSystemLoader

In [ ]:
# user input html form
template = Template('''
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html>
    <head>
        <title>Identification Classification</title>
        <link rel="stylesheet" media="screen" href ="bootstrap.min.css">
        <meta name="viewport" content = "width=device-width, initial-scale=1.0">
 
    </head>
    <body>
 
 
<div class="container">
 
 
  <h2>Identification Classification</h2>
    <form  action="http://localhost:5000/result" method="POST" role="form">
    {{ form.csrf }}
    <div class="form-group">
      <label for="address">Address:</label>
        <input type="text" class="form-control" id="address" name="address" placeholder="What's your address?">
        <br>
        <label for="algorithm">Algorithm:</label>
        <select name = "algorithm">
        <option value="K_Means">K Means</option>
        <option value="lda">LDA</option>
        <option value="HCA">HCA</option>
        <option value="gmm">GMM</option>
        </select>
        <br>
        <label for="cluster">Cluster:</label>
        <input type="text" class="form-control" id="cluster" name="cluster" placeholder="Enter a number.">
        <br>
 
    </div>
    <button type="submit" class="btn btn-success">Submit</button>
    </form>
 
    <br>
    
 
 </div>
<br>            
</div>
</div>
</body>
</html>
''')

In [ ]:
# result html form for image display
template2 = Template('''
<!doctype html>
<html>
   <body>
   
      <img src ="C:\Users\44070779\ward_clusters.png">
      
   </body>
</html>
''')

In [ ]:
# result html form for cluster display
template3 = Template('''
<!doctype html>
<head>
        <title>Here are your results</title>
</head>

<html>
    <body>
        <table border = 1>
         {% for line in result %}
             <tr><td>{{ line }}</td></tr>
            
         {% endfor %}
      </table>
   </body>
</html>
''')
#<td> {{ value }} </td>

In [ ]:
# App config.
DEBUG = True
app = Flask(__name__)
app.config.from_object(__name__)
app.config['SECRET_KEY'] = '7d441f27d441f27567d441f2b6176a'
 
class ReusableForm(Form):
    address = TextField('Address:', validators=[validators.required()])
    algorithm = TextField('Algorithm:', validators=[validators.required(), validators.Length(min=6, max=35)])
    cluster = TextField('Cluster:', validators=[validators.required(), validators.Length(min=3, max=35)])
 
 
@app.route("/", methods=['GET'])
def hello():
    form = ReusableForm(request.form)
    return render_template(template, form=form)

@app.route('/result',methods = ['POST', 'GET'])
def result():
    if request.method == 'POST':
        form = ReusableForm(request.form)
        result = request.form
        print form.errors
        address=request.form['address']
        algorithm=request.form['algorithm']
        cluster=request.form['cluster']
        print address, " ", algorithm, " ", cluster

        address = address.encode('ascii','ignore')
        algorithm = algorithm.encode('ascii','ignore')
        cluster = int(cluster)
        
        #if form.validate():
        # Save the comment here.
         #   flash('Thanks for registration ' + algorithm)
        #else:
        #    flash('Error: All the form fields are required. ')
        #func_name(address,algorithm,cluster)
        f_names = []
        # now we start loading the dataset
        filenames = os.listdir(address)
        for file in filenames:
            f_name = os.path.join(address, file)
            f_names.append(f_name)
    
        data_corpus = {}

        for f in filenames:
            f_name = os.path.join(address, f)
            fi = open(f_name)
            data = fi.read()
            data_corpus[f] = data
        data = pd.DataFrame.from_dict(data_corpus,orient='index')
        # pandas allows u to have a sneak peak at ur data
        data = data.reset_index()
        data.columns = ['file_name','text']
        
        # cleaning the data
        def strip(raw_text):
            letters_only = re.sub('[\s+]', " ", raw_text)
            return letters_only

        data['strip']= data['text'].apply(strip)
        
        #removing punctuations 
        def remove_punctuation(s):
            s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
            return s

        data['text_punc'] = data['strip'].apply(remove_punctuation)
        
        #removing numbers
        def numbers(raw_text):
            letters_only = re.sub(r'\d+', '', raw_text)
            return letters_only
        
        data['num'] = data['text_punc'].apply(numbers)
        
        #removing non eng words
        words = set(nltk.corpus.words.words())
        def non_eng_text(raw_text):
            wor = raw_text.split()
            meaningful_words = [w for w in wor \
                if w.lower() in words or not w.isalpha()]
            return( " ".join( meaningful_words ))
        
        data['non_eng']= data['num'].apply(non_eng_text)
        
        #converting to lowercase
        data['lower'] = data['non_eng'].str.lower()
        
        #removing stopwords
        def clean_text(raw_text):
            letters_only = re.sub("[^a-z]", " ", raw_text)
            words = letters_only.split()
            corp = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
        'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
        'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
        'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
        'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
        'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
        'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
        'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
        'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
        'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
        'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
        'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
            stops = set(corp)
            meaningful_words = [w for w in words if not w in stops and len(w)>1]
            return( " ".join( meaningful_words ))
        
        data['stop']= data['lower'].apply(clean_text)
        
        #lemmatizing the words
        def lemma(raw_text):
            wor = raw_text.split()
            wnl = WordNetLemmatizer()
            meaningful = [wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(wor)]
            return ( " ".join( meaningful))
        
        data['text_lemma']= data['stop'].apply(lemma)
        
        #creating tf-idf of the dataset available
        corp = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
        'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
        'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
        'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
        'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
        'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
        'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
        'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
        'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
        'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
        'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
        'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
        tfidf_vectorizer = TfidfVectorizer(max_features=500000, stop_words = corp, use_idf= True, ngram_range=(1,3))
        train_data_feature=tfidf_vectorizer.fit_transform(data['text_lemma'])
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        tf_vectorizer = CountVectorizer( max_features=500000, stop_words = corp)
        tf = tf_vectorizer.fit_transform(data['text_lemma'])
        tf_feature_names = tf_vectorizer.get_feature_names()
        train_data_norm = sklearn.preprocessing.normalize(train_data_feature)
        tf_norm = sklearn.preprocessing.normalize(tf)
        train_data_np = train_data_norm.toarray()
        tf_np = tf_norm.toarray()
        dist = 1 - cosine_similarity(train_data_feature)
        
        #checking which algorithm user has asked for
        if(algorithm == "HCA"):
            #titles of the document in the dataset
            titles = open('NAME.txt').read().split('\n')
            linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

            fig, ax = plt.subplots(figsize=(15, 20)) # set size
            ax = dendrogram(linkage_matrix, orientation="right", labels=titles); 
            #response= send_file(io.BytesIO(f.read()), minetype='image/png')
            
            plt.tick_params(\
            axis= 'x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='off')

            plt.tight_layout() #show plot with tight layout
            plt.savefig('ward_clusters.png', dpi=200)#save figure as ward_clusters
            return render_template(template2)
        elif(algorithm == "LDA"): 
                def strip_proppers(text):
            # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
                    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
                    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
                def strip_proppers_POS(text):
                    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
                    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
                    return non_propernouns
                stopwords = nltk.corpus.stopwords.words('english')
                stemmer = SnowballStemmer("english")
                def tokenize_and_stem(text):
                    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
                    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
                    filtered_tokens = []
                    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
                    for token in tokens:
                        if re.search('[a-zA-Z]', token):
                            filtered_tokens.append(token)
                    stems = [stemmer.stem(t) for t in filtered_tokens]
                    return stems
                preprocess = [strip_proppers(doc) for doc in data['stop']]

                tokenized_text = [tokenize_and_stem(text) for text in preprocess]

                texts = [[word for word in text if word not in stopwords] for text in tokenized_text]
                texts = [[word for word in text] for text in texts]

                dictionary = corpora.Dictionary(texts)
                corpus = [dictionary.doc2bow(text) for text in texts]
                lda = models.LdaModel(corpus, num_topics=cluster, id2word=dictionary, update_every=5, chunksize=10000, passes=100)
                topics = lda.print_topics(cluster, num_words=20)
                topics_matrix = lda.show_topics(formatted=False, num_words=20)
                topics_matrix_1 = np.array(topics_matrix, dtype=object)
                topic_words = topics_matrix_1[:,1]
                lda_corpus = lda[mm]
                #compute threshold 
                scores = list(chain(*[[score for topic_id,score in topic] \
                          for topic in [doc for doc in lda_corpus]]))
                threshold = sum(scores)/len(scores)
                lda_corpus = [max(prob,key=lambda y:y[1]) for prob in lda[mm] ]
                playlists = [[] for i in xrange(5)]
                a = []
                #returning top 20 words for each cluster formed
                for i in topic_words:
                    b = ""
                    for word in i:
                        b = b + str(word) + ", "  
                    a.append(b)
                print a
                return render_template(template3, result=a)
                #for i, x in enumerate(lda_corpus):
                    #playlists[x[0]].append(data['file_name'][i])
                #for idx in range (cluster):
                    ##newpath = ((r'C:\Users\44070779\Desktop\folder_%s') % (idx)) 
                    #if not os.path.exists(newpath): os.makedirs(newpath)
        elif(algorithm == "K_Means"):
            print "K-Means"
            km =  KMeans(n_clusters=cluster)
            km.fit(train_data_feature)
            clusters = km.labels_.tolist()
            data['cluster'] = clusters
            km_model = []
            '''for i in xrange(2,8):
                print ("Running Kmeans with", i , "clusters")
                km_model.append(KMeans(n_clusters=i).fit(train_data_norm))
            def print_file_with_clusters(cluster):
                file_with_clusters = {}
                data['cluster'] = km_model[cluster - 2].labels_

                for i in xrange(cluster):
                    file_with_clusters[i] = list(data[data['cluster']==i]['file_name'])

                pprint.pprint(file_with_clusters)'''
            order_centroids = km.cluster_centers_.argsort()[:, ::-1]
            a = []
            #returning top 20 words for each cluster formed
            for i in range(cluster):
                b = ""
                for ind in order_centroids[i, :20]:
                    b = b + tfidf_feature_names[ind] + ", "  
                a.append(b)
            print a
            return render_template(template3, result=a)

In [ ]:
#making the app run
if __name__ == "__main__":
    app.run(debug=False)
#wait for the magic ***